# Data Envelopment Analysis

In [1]:
Inputs = [:labor, :uranium]
Outputs = [:energy, :plutonium, :three_eyed_fish]
Units = [:plant1, :plant2, :plant3]

using NamedArrays
Unit_Input = [
    5 14
    8 15
    7 12
]
UI = NamedArray(Unit_Input, (Units,Inputs),("Units","Inputs"))
Unit_Output = [
    9 4 16
    5 7 10
    4 9 13
]
UO = NamedArray(Unit_Output, (Units,Outputs),("Units","Outputs"))

using JuMP, HiGHS
m = Model(HiGHS.Optimizer)
set_silent(m)

@variable(m, w[Inputs] >= 0)
@variable(m, t[Outputs] >= 0)


@expression(m, output_val[u in Units], sum(UO[u,j]*t[j] for j in Outputs))

@expression(m, input_val[u in Units], sum(UI[u,i]*w[i] for i in Inputs))

@constraint(m, [u in Units], output_val[u] - input_val[u] <= 0)

# Solve an optimization model for each unit to determine if it is efficient
for u in Units
    @objective(m, Max, output_val[u])
    @constraint(m, input_scaling, input_val[u] == 1)
    optimize!(m)
    println(u, " has efficiency ", objective_value(m))
    delete(m, input_scaling)
    unregister(m, :input_scaling)
end




plant1 has efficiency 1.0
plant2 has efficiency 0.7733333333333334
plant3 has efficiency 1.0
